In [1]:
import csv  
import json  
import os
import pandas as pd
from pprint import pprint

In [2]:
#Import and read in JSON file
file = os.path.join("NBA Salaries 2000-2019.json")
data = json.loads(open(file).read())

#print test data
pprint(data[0])

#Convert it so I am able to put it in a df
nbaSalaries = json.dumps(data)

{'ID': '1',
 'Name': "Shaquille O'Neal",
 'Rank': '1',
 'Salary': '17142000',
 'Year': '2000'}


In [3]:
#Store NBA Salaries in df
salaries_df = pd.read_json(nbaSalaries)
salaries_df.head()

,ID,Name,Rank,Salary,Year
0,1,Shaquille O'Neal,1,17142000,2000
1,2,Kevin Garnett,2,16806000,2000
2,3,Alonzo Mourning,3,15004000,2000
3,4,Juwan Howard,4,15000000,2000
4,5,Scottie Pippen,5,14795000,2000


In [4]:
#Import NBA Draft tsv file
tsv = os.path.join("NBA Draft 1980-2017.tsv")

#Store data in df
draft_df = pd.read_csv(tsv, sep='\t')
draft_df = draft_df.rename(columns={"year": "Draft Year", "round": "Draft Round", "pick": "Draft Pick Position", 
                                    "player": "Player", "position": "Position", "school": "School"})
#Pull out data from Year 2000 and up only
# draft_df = draft_df.loc[draft_df["Draft Year"] >= 2000].reset_index(drop=True)
draft_df.head()

,Draft Year,Draft Round,Draft Pick Position,Player,Position,School
0,1980,1,1.0,"Carroll, Joe Barry",F/C,Purdue
1,1980,1,2.0,"Griffith, Darrell",G,Louisville
2,1980,1,3.0,"McHale, Kevin",F/C,Minnesota
3,1980,1,4.0,"Ransey, Kelvin",G,Ohio State
4,1980,1,5.0,"Ray, James",F,Jacksonville


In [5]:
#Arrange the name of the draft data to read First + Last Name instead of Last + ", " + First name
fix_name = draft_df["Player"].str.split(" ", n = 1, expand = True)
fix_name = fix_name.rename(columns={1: "First Name", 0: "Last Name"})
fix_name["Name"] = fix_name["First Name"] + " " + fix_name["Last Name"]
fix_name = fix_name[["Name", "First Name", "Last Name"]]
fix_name["Name"] = fix_name["Name"].str.replace("," , "")
fix_name["Last Name"] = fix_name["Last Name"].str.replace("," , "")
fix_name.head()

,Name,First Name,Last Name
0,Joe Barry Carroll,Joe Barry,Carroll
1,Darrell Griffith,Darrell,Griffith
2,Kevin McHale,Kevin,McHale
3,Kelvin Ransey,Kelvin,Ransey
4,James Ray,James,Ray


In [6]:
#Merged fixed names back to the draft df
fixeddraft_df = pd.concat([draft_df, fix_name], axis=1)
fixeddraft_df.head()

,Draft Year,Draft Round,Draft Pick Position,Player,Position,School,Name,First Name,Last Name
0,1980,1,1.0,"Carroll, Joe Barry",F/C,Purdue,Joe Barry Carroll,Joe Barry,Carroll
1,1980,1,2.0,"Griffith, Darrell",G,Louisville,Darrell Griffith,Darrell,Griffith
2,1980,1,3.0,"McHale, Kevin",F/C,Minnesota,Kevin McHale,Kevin,McHale
3,1980,1,4.0,"Ransey, Kelvin",G,Ohio State,Kelvin Ransey,Kelvin,Ransey
4,1980,1,5.0,"Ray, James",F,Jacksonville,James Ray,James,Ray


In [7]:
#Names appear in varios orders, this first merge focuses on names originally in First Last name order in salary df
merge1 = fixeddraft_df[["Draft Year", "Draft Round", "Draft Pick Position", "Player", "Position"
                       , "School", "First Name", "Last Name"]]
merge1 = merge1.rename(columns={"Player": "Name"})
merge1 = pd.merge(salaries_df, merge1, how = "left", on = "Name").dropna()
merge1 = merge1[["Name", "Rank", "Salary", "Year", "Draft Year", "Draft Round", 
                                      "Draft Pick Position", "Position", "School"]]

merge1 = merge1.loc[merge1["Salary"] != "NA"].reset_index(drop=True)
merge1.head()

,Name,Rank,Salary,Year,Draft Year,Draft Round,Draft Pick Position,Position,School
0,Shaquille O'Neal,1,17142000,2000,1992.0,1.0,1.0,C,LSU
1,Alonzo Mourning,3,15004000,2000,1992.0,1.0,2.0,C,Georgetown
2,Gary Payton,8,11020000,2000,1990.0,1.0,2.0,PG,Oregon State
3,Shawn Kemp,10,10780000,2000,1989.0,1.0,17.0,PF/C,Trinity Valley CC
4,Vlade Divac,16,8837000,2000,1989.0,1.0,26.0,C,KK Partizan


In [8]:
#This second merge focuses on names that were previously converted
salaryANDdraft_df = pd.merge(salaries_df, fixeddraft_df, how = "left", on = "Name").dropna().reset_index(drop = True)
salaryANDdraft_df = salaryANDdraft_df[["Name", "Rank", "Salary", "Year", "Draft Year", "Draft Round", 
                                      "Draft Pick Position", "Position", "School"]]
salaryANDdraft_df = salaryANDdraft_df.loc[salaryANDdraft_df["Salary"] != "NA"].reset_index(drop=True)
salaryANDdraft_df.head()

,Name,Rank,Salary,Year,Draft Year,Draft Round,Draft Pick Position,Position,School
0,Kevin Garnett,2,16806000,2000,1995.0,1.0,5.0,PF,Farragut Academy HS
1,Juwan Howard,4,15000000,2000,1994.0,1.0,5.0,PF,Michigan
2,Scottie Pippen,5,14795000,2000,1987.0,1.0,5.0,SF,Central Arkansas
3,Karl Malone,6,14000000,2000,1985.0,1.0,13.0,PF,Louisiana Tech
4,Larry Johnson,7,11910000,2000,1991.0,1.0,1.0,PF,UNLV


In [9]:
#Concat the two merged tables one on top of another
final_df = pd.concat([merge1, salaryANDdraft_df], axis = 0).reset_index(drop=True)
final_df.head()

,Name,Rank,Salary,Year,Draft Year,Draft Round,Draft Pick Position,Position,School
0,Shaquille O'Neal,1,17142000,2000,1992.0,1.0,1.0,C,LSU
1,Alonzo Mourning,3,15004000,2000,1992.0,1.0,2.0,C,Georgetown
2,Gary Payton,8,11020000,2000,1990.0,1.0,2.0,PG,Oregon State
3,Shawn Kemp,10,10780000,2000,1989.0,1.0,17.0,PF/C,Trinity Valley CC
4,Vlade Divac,16,8837000,2000,1989.0,1.0,26.0,C,KK Partizan


In [10]:
#Format numerical values
final_df["Salary"] = pd.to_numeric(final_df["Salary"])
final_df["Salary"] = final_df["Salary"].map("${:,.0f}".format)
final_df["Draft Year"] = final_df["Draft Year"].map("{:.0f}".format)
final_df["Draft Round"] = final_df["Draft Round"].map("{:.0f}".format)
final_df["Draft Pick Position"] = final_df["Draft Pick Position"].map("{:.0f}".format)
final_df.head()

,Name,Rank,Salary,Year,Draft Year,Draft Round,Draft Pick Position,Position,School
0,Shaquille O'Neal,1,"$17,142,000",2000,1992,1,1,C,LSU
1,Alonzo Mourning,3,"$15,004,000",2000,1992,1,2,C,Georgetown
2,Gary Payton,8,"$11,020,000",2000,1990,1,2,PG,Oregon State
3,Shawn Kemp,10,"$10,780,000",2000,1989,1,17,PF/C,Trinity Valley CC
4,Vlade Divac,16,"$8,837,000",2000,1989,1,26,C,KK Partizan


In [11]:
#Sort by Salary Year and the Salary Rank
final_df = final_df.sort_values(["Year", "Rank"], ascending=[True, True]).reset_index(drop=True)
final_df.head()

,Name,Rank,Salary,Year,Draft Year,Draft Round,Draft Pick Position,Position,School
0,Shaquille O'Neal,1,"$17,142,000",2000,1992,1,1,C,LSU
1,Kevin Garnett,2,"$16,806,000",2000,1995,1,5,PF,Farragut Academy HS
2,Alonzo Mourning,3,"$15,004,000",2000,1992,1,2,C,Georgetown
3,Juwan Howard,4,"$15,000,000",2000,1994,1,5,PF,Michigan
4,Scottie Pippen,5,"$14,795,000",2000,1987,1,5,SF,Central Arkansas


In [12]:
#Write df of 1000 rows to excel
to_excel = final_df.head(1000)
to_excel.to_csv('NBA Salary and Draft Year Data.csv', index = False)